In [310]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime

In [311]:
def find_longest_falling_streak(file_path, start_date=None, end_date=None, green_candles_limit=5):
    # Загрузка данных из CSV файла
    df = pd.read_csv(file_path, parse_dates=['Date'])
    
    # Фильтрация по заданным датам
    if start_date:
        df = df[df['Date'] >= pd.to_datetime(start_date)]
        
    if end_date:
        df = df[df['Date'] <= pd.to_datetime(end_date)]

    # Добавляем колонку для определения падения (True, если свеча красная, иначе False)
    df['is_red'] = df['Close'] < df['Open']
    
    # Колонка для небольших зеленых свечей (по изменению цены)
    df['small_green'] = (df['Close'] > df['Open']) & (df['Close'] - df['Open'] < 0.01 * df['Open'])  # Задать порог для "небольшой свечи"
    
    # Вспомогательные переменные
    max_streak = 0
    current_streak = 0
    green_candles_in_streak = 0
    streak_start_index = None
    max_streak_info = {"start_date": None, "end_date": None, "length": 0}

    for i, row in df.iterrows():
        if row['is_red'] or (row['small_green'] and green_candles_in_streak < green_candles_limit):
            # Начало новой последовательности падения
            if current_streak == 0 and row['is_red']:
                streak_start_index = i

            if current_streak > 0 or row['is_red']:  # Продолжение только если началась с красной свечи
                current_streak += 1
                if row['small_green']:
                    green_candles_in_streak += 1
        else:
            # Проверяем максимальную последовательность, если она начинается и заканчивается красной свечой
            if current_streak > 0 and df.loc[i - 1, 'is_red']:
                if current_streak > max_streak:
                    max_streak = current_streak
                    max_streak_info['start_date'] = df.loc[streak_start_index, 'Date']
                    max_streak_info['end_date'] = df.loc[i - 1, 'Date']
                    max_streak_info['length'] = max_streak

            current_streak = 0
            green_candles_in_streak = 0

    # Если последняя последовательность оказалась самой длинной
    if current_streak > 0 and df.loc[i, 'is_red']:
        if current_streak > max_streak:
            max_streak_info['start_date'] = df.loc[streak_start_index, 'Date']
            max_streak_info['end_date'] = df.iloc[-1]['Date']
            max_streak_info['length'] = current_streak

    # Вывод результатов
    print(f"Самая длинная последовательность падений: {max_streak_info['length']} свечей")
    print(f"Начало: {max_streak_info['start_date']}, Конец: {max_streak_info['end_date']}")
    
    # Визуализация графика свечей с выделенным участком
    fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                                         open=df['Open'],
                                         high=df['High'],
                                         low=df['Low'],
                                         close=df['Close'])])
    
    # Добавляем красный фон на период падения
    if max_streak_info['start_date'] and max_streak_info['end_date']:
        fig.add_vrect(
            x0=max_streak_info['start_date'], x1=max_streak_info['end_date'],
            fillcolor="red", opacity=0.3,
            layer="below", line_width=0)

    fig.update_layout(
        title="График актива с максимальным падением",
        xaxis_title="Дата",
        yaxis_title="Цена",
        xaxis_rangeslider_visible=False)
    
    fig.show()

In [312]:
def find_longest_falling_streak_2(df, thres_val = 0.005, start_date=None, end_date=None, green_candles_limit=5):
    if start_date:
        df = df[df['Date'] >= pd.to_datetime(start_date)]
        
    if end_date:
        df = df[df['Date'] <= pd.to_datetime(end_date)]

    current_streak = 0
    current_prc = 0
    max_prc = 0
    green_candles_in_streak = 0
    streak_start_index = None
    max_streak_info = {"start_date": [], "end_date": [], "percentage": []}

    for i, row in df.iterrows():
        if row['Price_Change'] < 0:
            if current_streak == 0:
                current_streak = 1
                streak_start_index = i
                current_prc = row['Price_Change']
                print(f"Начался Новый стрик индекс {i} падение {current_prc}")
            
            else:
                current_streak += 1
                current_prc += row['Price_Change']
                print(f"Продолжаем стрик индекс {i}, падение {current_prc}")
        
        elif row['Price_Change'] < thres_val and green_candles_in_streak < green_candles_limit:
            green_candles_in_streak += 1
            if current_streak == 0:
                current_streak = 1
                streak_start_index = i
                current_prc = row['Price_Change']
                print(f"Начался Новый стрик индекс {i} падение {current_prc}")
            
            else:
                current_streak += 1
                current_prc += row['Price_Change']
                print(f"Продолжаем стрик индекс {i}, падение {current_prc}")
        
        else:
            if current_streak > 0 and df.loc[i - 1, 'Price_Change'] < 0:
                max_streak_info['start_date'].append(df.loc[streak_start_index, 'Date'])
                max_streak_info['end_date'].append(df.loc[i - 1, 'Date'])
                max_streak_info['percentage'].append(current_prc)
                
                print(f"Стрик закончился индекс {i}, Максимальное падение: {max_prc}")
                current_streak = 0
                current_prc = 0
                green_candles_in_streak = 0

    # Если последняя последовательность оказалась самой длинной
    if current_streak > 0 and df.loc[i - 1, 'Price_Change'] < 0:
        max_streak_info['start_date'].append(df.loc[streak_start_index, 'Date'])
        max_streak_info['end_date'].append(df.loc[i - 1, 'Date'])
        max_streak_info['percentage'].append(current_prc)
    
    return max_streak_info

In [313]:
file_path = "D:\\PythonScripts\\RL_for_Trading\\data\\somthing\\PEPE_per_minute_5_2024_10_19.csv"
df_prices = pd.read_csv(file_path, parse_dates=['Date'])
df_prices['Price_Change'] = ((df_prices['Close'] - df_prices['Open']) / df_prices['Open']) * 100

In [314]:
df_prices

,Date,Open,High,Low,Average,Close,Change,Volume,Volume Change,Price_Change
0,2024-10-17 11:25:00,0.000010,0.000010,0.000010,0.000010,0.000010,0.000000,520.184654,0.000000,0.000000
1,2024-10-17 11:35:00,0.000010,0.000010,0.000010,0.000010,0.000010,-0.035695,297.543664,-42.800376,-0.029919
2,2024-10-17 11:40:00,0.000010,0.000010,0.000010,0.000010,0.000010,-0.016980,1779.172481,497.953408,-0.019952
3,2024-10-17 11:55:00,0.000010,0.000010,0.000010,0.000010,0.000010,-0.669684,89.825209,-94.951293,-0.668529
4,2024-10-17 12:00:00,0.000010,0.000010,0.000010,0.000010,0.000010,0.574022,2976.792169,3213.983016,0.572577
...,...,...,...,...,...,...,...,...,...,...
283,2024-10-18 20:30:00,0.000011,0.000011,0.000011,0.000011,0.000011,-0.473081,1225.363256,-98.111125,-0.473754
284,2024-10-18 20:35:00,0.000011,0.000011,0.000011,0.000011,0.000011,-0.014502,2849.433975,132.537899,-0.009520
285,2024-10-18 20:45:00,0.000011,0.000011,0.000010,0.000010,0.000010,-0.099399,168.357470,-94.091547,-0.104732
286,2024-10-18 20:50:00,0.000010,0.000011,0.000010,0.000011,0.000011,0.595487,1194.028397,609.222108,0.600457


In [315]:
streak_info = find_longest_falling_streak_2(df_prices, thres_val = 0.2, green_candles_limit= 5)

Начался Новый стрик индекс 0 падение 0.0
Продолжаем стрик индекс 1, падение -0.029919218111114945
Продолжаем стрик индекс 2, падение -0.049871333035289955
Продолжаем стрик индекс 3, падение -0.718400568716786
Стрик закончился индекс 4, Максимальное падение: 0
Начался Новый стрик индекс 5 падение 0.08989212944466046
Продолжаем стрик индекс 6, падение 0.1996616135280893
Продолжаем стрик индекс 7, падение 0.3491831446285748
Продолжаем стрик индекс 8, падение 0.3889960240950852
Продолжаем стрик индекс 9, падение 0.488488611897293
Продолжаем стрик индекс 12, падение -0.04674778417763836
Продолжаем стрик индекс 13, падение -0.0666780283231222
Продолжаем стрик индекс 14, падение -0.08661224540675617
Стрик закончился индекс 15, Максимальное падение: 0
Начался Новый стрик индекс 16 падение -0.04955892556248494
Продолжаем стрик индекс 17, падение -0.3867267161217869
Продолжаем стрик индекс 18, падение -0.7648361688581085
Стрик закончился индекс 19, Максимальное падение: 0
Начался Новый стрик инд

In [316]:
streak_info_df = pd.DataFrame(data = streak_info)

In [317]:
streak_info_df = streak_info_df.sort_values(by = "percentage")
streak_info_df.head()

,start_date,end_date,percentage
34,2024-10-18 10:40:00,2024-10-18 12:35:00,-2.656315
8,2024-10-17 17:15:00,2024-10-17 17:35:00,-2.372365
24,2024-10-18 02:35:00,2024-10-18 03:20:00,-1.649344
45,2024-10-18 16:45:00,2024-10-18 17:20:00,-1.638470
16,2024-10-17 20:30:00,2024-10-17 21:55:00,-1.618863


In [318]:
# Создание свечного графика на основе df_prices
candlestick = go.Candlestick(
    x=df_prices['Date'],
    open=df_prices['Open'],
    high=df_prices['High'],
    low=df_prices['Low'],
    close=df_prices['Close'],
    name='Price',
)

# Теперь добавим красные зоны для падений из второго DataFrame df_drops
drop_zones = []
for index, row in streak_info_df.iterrows():
    drop_zones.append(
        go.Scatter(
            x=[row['start_date'], row['end_date'], row['end_date'], row['start_date'], row['start_date']],
            y=[df_prices['High'].max(), df_prices['High'].max(), df_prices['Low'].min(), df_prices['Low'].min(), df_prices['High'].max()],
            fill='toself',
            fillcolor='rgba(255, 0, 0, 0.3)',  # Прозрачный красный цвет
            line=dict(color='rgba(255, 0, 0, 0)'),
            hoverinfo='skip',  # Отключаем отображение при наведении
            showlegend=False
        )
    )

# Объединяем все данные в одну фигуру
fig = go.Figure(data=[candlestick] + drop_zones)

# Настройки отображения
fig.update_layout(
    title='Stock Prices with Drop Zones',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False
)

# Отображаем график
fig.show()